In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
from tensorflow.keras.models import load_model
model = load_model('../model_saved/timeseries_l50_v2.h5')

In [ ]:
import hls4ml
import os
os.environ['PATH'] = '/opt/Xilinx/Vivado/2020.1/bin:' + os.environ['PATH']

In [ ]:
i = 3  # fraction bits
reuse = 4
int_bit = 7


precision = 'ap_fixed<{},{}>'.format((int_bit+i), int_bit)
fix_int_precision = 'ap_fixed<{},{}>'.format((16+i), 14)
dirc = 'runs/CE_run/r{}_{}int_{}frac'.format(reuse, int_bit, i)
#First, the baseline model
hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')

# Set the precision and reuse factor for the full model
hls_config['Model']['Precision'] = precision
hls_config['Model']['ReuseFactor'] = reuse
hls_config['Model']['Strategy'] = 'Resource'

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Precision'] = precision
    hls_config['LayerName'][Layer]['Strategy'] = 'Resource'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1
    hls_config['LayerName'][Layer]['weight'] = precision
    hls_config['LayerName'][Layer]['scale'] = precision
    hls_config['LayerName'][Layer]['bias'] = precision
    hls_config['LayerName'][Layer]['accum_t'] = 'ap_fixed<22,10>'
    if 'multi_head_attention' in Layer:
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['exp_range'] = 4
        hls_config['LayerName'][Layer]['table_size'] = 1024
    if 'dense' in Layer:
        hls_config['LayerName'][Layer]['inv_range'] = 256
        hls_config['LayerName'][Layer]['table_size'] = 1024
        hls_config['LayerName'][Layer]['exp_range'] = 4

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_parallel' 
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = dirc
cfg['Part'] = 'xcvu13p-fhga2104-2L-e'

hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()
hls_model.build(csim=False, synth=True, vsynth=True)